In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import shapely
import rasterio as rio
import numba as nb
from matplotlib.colors import ListedColormap
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
import rasterio.plot as rioplot

In [ ]:

brazil_gdf = gpd.read_file('/home/ksolvik/research/reservoirs/analysis/data/misc/general_borders/Brazilian_States_aea.shp')
brazil_gdf = brazil_gdf.to_crs('EPSG:4326')

mt_gdf = brazil_gdf.loc[brazil_gdf['UF_05'] == 'MT']
mt_gdf = mt_gdf.to_crs('EPSG:4326')

In [ ]:
mt_gdf.plot()

In [ ]:
all_csvs = glob.glob('../clean_summarize/out/v3_cloudfilt/ls*wgs84_merged.csv')
all_csvs.sort()
# Only the first and last yea
all_csvs = ['../clean_summarize/out/v3_cloudfilt/ls5_1985_cloudfilt_v3_wgs84_merged.csv', 
            '../clean_summarize/out/v3_cloudfilt/ls7_2019_cloudfilt_v3_wgs84_merged.csv' ]

In [ ]:

def read_process_csv(csv):
    temp_df = pd.read_csv(csv)
    temp_df['satellite'] = os.path.basename(csv)[:3]
    temp_df['year'] = int(os.path.basename(csv)[4:8])
    return temp_df

In [ ]:
full_df = pd.concat([
    read_process_csv(csv) for csv in all_csvs
])

full_df = full_df.loc[full_df['hydropoly_max']<100]

full_df['area'] = full_df['area']*100/10000
full_df = full_df.loc[full_df['area']<100]

In [ ]:
xlims = mt_gdf.bounds.min()['minx'], mt_gdf.bounds.max()['maxx']
xlims = (xlims[0] - 0.25, xlims[1]+0.25)
ylims = mt_gdf.bounds.min()['miny'], mt_gdf.bounds.max()['maxy']
ylims = (ylims[0] - 0.25, ylims[1] + 0.25)

In [ ]:

%matplotlib inline

axes_height_ratios=[1, 0.05]
fig, axs = plt.subplots(2, 2, figsize = (7.35, 5),
                       gridspec_kw={"height_ratios":axes_height_ratios})

# Plot 1984 first
# brazil_gdf.boundary.plot(ax=axs[0, 0], color='grey', alpha=0.3, lw=0.7)
# brazil_gdf.boundary.plot(ax=axs[0, 1], color='grey', alpha=0.3, lw=0.7)
mt_gdf.boundary.plot(ax=axs[0, 0], color='black', alpha=0.8, lw=0.9)
mt_gdf.boundary.plot(ax=axs[0, 1], color='black', alpha=0.8, lw=0.9)


outline_gdf = gpd.GeoDataFrame(
    geometry=gpd.GeoSeries(shapely.geometry.Polygon(
        [[xlims[0], ylims[0]],
         [xlims[0], ylims[1]],
         [xlims[1], ylims[1]],
         [xlims[1], ylims[0]],
         [xlims[0], ylims[0]]])),
    crs='EPSG:4326')
nonmt_poly = outline_gdf.overlay(mt_gdf, how='difference')


# Plot start
first_df = full_df.loc[full_df['year']==1985]
axs[0,0].hexbin(first_df['longitude'], first_df['latitude'],
              gridsize=(50),
             vmin=0, vmax=275,
             extent=xlims + ylims,
             cmap='GnBu',
             linewidths=0.1,
             edgecolor='none')
             

# Plot end
last_df = full_df.loc[full_df['year']==2019]
im = axs[0, 1].hexbin(last_df['longitude'], last_df['latitude'],
              gridsize=(50),
             vmin=0, vmax=275,
             extent=xlims + ylims,
             cmap='GnBu',
             linewidths=0.1,
             edgecolor='none')

# fig.colorbar(im)

for cur_ax in axs[0]:
    cur_ax.set_xlabel('Longitude (deg)')
    cur_ax.set_ylabel('Latitude (deg)')
    cur_ax.set_xlim(xlims)
    cur_ax.set_ylim(ylims)
axs[0,0].set_title('1984', size=12)
axs[0,1].set_title('2019', size=12)

# Remove outside of mt
nonmt_poly.plot(ax=axs[0,1],color='white')
nonmt_poly.plot(ax=axs[0,0],color='white')

# Set up colorbar

# Setting this to be per area required some fiddling
print('Max value:',im.get_array().data.max())
vmax = im.get_clim()[1]
tick_increment = vmax/5
hex_area = (outline_gdf.to_crs('ESRI:102033').area.values[0]/len(im.get_array()))/(1000*1000)
print('Hex area:', hex_area)
print('Max cbar density:', vmax/hex_area)

gs = axs[0, 0].get_gridspec()
for ax in axs[-1]:
    ax.remove()
axbig = fig.add_subplot(gs[-1, :])
axbig.set_title('Reservoir density (# per km$^2)$')
cb = fig.colorbar(im, cax=axbig, orientation='horizontal',
                  ticks=np.arange(0, vmax+1, tick_increment))
cb.ax.set_xticklabels([0, 0.1, 0.2, 0.3, 0.4, '$\geq$0.5'])
fig.tight_layout()

In [ ]:
# MapBiomas Map
mb_keys_dict = {
    'crop': np.array([18,19,39,20,40,62,41,36,46,47,35,48]),
    'natural': np.array([3, 4, 12]),
    'pasture': np.array([15])
}
new_val_dict =  {
    'crop':3,
    'natural':1,
    'pasture':2
}

In [ ]:
@nb.njit(parallel=True)
def nb_isin_listcomp(matrix, index_to_remove):
    #matrix and index_to_remove have to be numpy arrays
    #if index_to_remove is a list with different dtypes this 
    #function will fail
    og_shape = matrix.shape
    matrix = matrix.flatten()
    out=np.empty(matrix.shape[0],dtype=nb.boolean)
    index_to_remove_set=set(index_to_remove)

    for i in nb.prange(matrix.shape[0]):
        if matrix[i] in index_to_remove_set:
            out[i]=True
        else:
            out[i]=False

    return out.reshape(og_shape)


def calc_mask(ar, cover):
    mask = nb_isin_listcomp(ar, mb_keys_dict[cover])
    return mask

def make_cover_ar(y):
    # og_fp =  './in/mato_grosso/wgs84/mapbiomas-brazil-collection-90-matogrossomt-{}.tif'.format(y)
    og_fp =  './in/mato_grosso/wgs84/mapbiomas-brazil-collection-90-matogrossomt-{}_1km_mode.tif'.format(y)
    og_ar = rio.open(og_fp).read()

    # Write out values
    out_ar = np.zeros_like(og_ar)
    # "Other"
    out_ar[og_ar>0]=4
    for cover in mb_keys_dict.keys():
        mask = calc_mask(og_ar, cover)
        out_ar[mask] = new_val_dict[cover]
        print(mask.sum())
            
    return out_ar


In [ ]:
cover_2019 = make_cover_ar(2019)
cover_1985 = make_cover_ar(1985)



In [ ]:
cmap = ListedColormap(["white", "darkgreen", "sienna", "pink", "slategrey"])
# cmap = ListedColormap(['white','#1f8d40','#c7b27b','#f39ab6', 'slajjk

In [ ]:
# Improvements: https://rasterio.readthedocs.io/en/stable/topics/plotting.html

In [ ]:

%matplotlib inline

fig, axs = plt.subplots(1, 2, figsize = (7.35, 5))

# Plot 1984 first
# mt_gdf.boundary.plot(ax=axs[0, 0], color='red', alpha=0.3)



src = rio.open('./in/mato_grosso/wgs84/mapbiomas-brazil-collection-90-matogrossomt-1985_1km_mode.tif')

rioplot.show(cover_1985[0], 
             vmin=0, vmax=4,
             cmap=cmap, transform=src.transform, ax=axs[0])
rioplot.show(cover_2019[0], 
             vmin=0, vmax=4,
             cmap=cmap, transform=src.transform, ax=axs[1])
for cur_ax in axs:
    cur_ax.set_xlabel('Longitude (deg)')
    cur_ax.set_ylabel('Latitude (deg)')
axs[0].set_title('1985', size=14)
axs[1].set_title('2019', size=14)

# Combined plot


In [ ]:

axes_height_ratios=[1, 1, 0.05]
fig, axs = plt.subplots(3, 2, figsize = (7.35, 7.5),
                       gridspec_kw={"height_ratios":axes_height_ratios})

# First rio plots
src = rio.open('./in/mato_grosso/wgs84/mapbiomas-brazil-collection-90-matogrossomt-1985_1km_mode.tif')

rioplot.show(cover_1985[0], 
             vmin=0, vmax=4,
             cmap=cmap, transform=src.transform, ax=axs[0, 0])
rioplot.show(cover_2019[0], 
             vmin=0, vmax=4,
             cmap=cmap, transform=src.transform, ax=axs[0,1])

mt_gdf.boundary.plot(ax=axs[0, 0], color='black', alpha=0.8, lw=0.9)
mt_gdf.boundary.plot(ax=axs[0, 1], color='black', alpha=0.8, lw=0.9)
axs[0,0].set_title('1985', size=12)
axs[0,1].set_title('2019', size=12)

# -------------- Then res plots --------------
# Plot 1984 first
mt_gdf.boundary.plot(ax=axs[1, 0], color='black', alpha=0.8, lw=0.9)
mt_gdf.boundary.plot(ax=axs[1, 1], color='black', alpha=0.8, lw=0.9)
outline_gdf = gpd.GeoDataFrame(
    geometry=gpd.GeoSeries(shapely.geometry.Polygon(
        [[xlims[0], ylims[0]],
         [xlims[0], ylims[1]],
         [xlims[1], ylims[1]],
         [xlims[1], ylims[0]],
         [xlims[0], ylims[0]]])),
    crs='EPSG:4326')
nonmt_poly = outline_gdf.overlay(mt_gdf, how='difference')


# Plot start
first_df = full_df.loc[full_df['year']==1985]
axs[1,0].hexbin(first_df['longitude'], first_df['latitude'],
              gridsize=(50),
             vmin=0, vmax=275,
             extent=xlims + ylims,
             cmap='GnBu',
             linewidths=0.1,
             edgecolor='none')
             

# Plot end
last_df = full_df.loc[full_df['year']==2019]
im = axs[1, 1].hexbin(last_df['longitude'], last_df['latitude'],
              gridsize=(50),
             vmin=0, vmax=275,
             extent=xlims + ylims,
             cmap='GnBu',
             linewidths=0.1,
             edgecolor='none')

# fig.colorbar(im)

for cur_ax in axs[:2].flatten():
    cur_ax.set_xlabel('Lon (deg)')
    cur_ax.set_ylabel('Lat (deg)')
    cur_ax.set_xlim(xlims)
    cur_ax.set_ylim(ylims)
    # Remove outside of mt
    nonmt_poly.plot(ax=cur_ax,color='whitesmoke')


# Set up colorbar

# Setting this to be per area required some fiddling
print('Max value:',im.get_array().data.max())
vmax = im.get_clim()[1]
tick_increment = vmax/5
hex_area = (outline_gdf.to_crs('ESRI:102033').area.values[0]/len(im.get_array()))/(1000*1000)
print('Hex area:', hex_area)
print('Max cbar density:', vmax/hex_area)

gs = axs[2, 0].get_gridspec()
for ax in axs[-1]:
    ax.remove()
axbig = fig.add_subplot(gs[-1, :])
axbig.set_title('Reservoir density (# per km$^2)$')
cb = fig.colorbar(im, cax=axbig, orientation='horizontal',
                  ticks=np.arange(0, vmax+1, tick_increment))
cb.ax.set_xticklabels([0, 0.1, 0.2, 0.3, 0.4, '$\geq$0.5'])
# Add abcd labels
for i, label in enumerate(['A', 'B', 'C', 'D']):
    axs[:2].flatten()[i].annotate(
            label,
            xy=(0, 1), xycoords='axes fraction',
            xytext=(0.25, -1.3), textcoords='offset fontsize',
            fontsize=12, verticalalignment='bottom', fontfamily='serif')
fig.tight_layout(pad=-0.5)
plt.savefig('/home/ksolvik/research/reservoirs/figs/ch2/res_dist_vs_lulc.svg', dpi=300)
plt.savefig('/home/ksolvik/research/reservoirs/figs/ch2/res_dist_vs_lulc.jpg', dpi=300,
            pil_kwargs={'quality':95},
            bbox_inches='tight')

# GIF

In [ ]:
all_cover = {y:make_cover_ar(y) for y in range(1985, 2020)}

In [ ]:
all_csvs = glob.glob('../clean_summarize/out/v3_cloudfilt/ls*wgs84_merged.csv')
all_csvs.sort()
full_df = pd.concat([
    read_process_csv(csv) for csv in all_csvs
])

full_df = full_df.loc[full_df['hydropoly_max']<100]

full_df['area'] = full_df['area']*100/10000
full_df = full_df.loc[full_df['area']<100]

In [ ]:
full_df = full_df.loc[full_df['satellite']!='ls8']
full_df = full_df.loc[~((full_df['satellite']=='ls5') & (full_df['year']>2000))]
full_df = full_df.loc[~((full_df['satellite']=='ls7') & (full_df['year']==2000))]

In [ ]:

%matplotlib notebook

fig, axs = plt.subplots(1,2, figsize=(10,5))
for ax in axs:
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax.set_xlabel('Lon (deg)')
    ax.set_ylabel('Lat (deg)')
    mt_gdf.boundary.plot(ax=ax, color='black', alpha=0.8, lw=0.9)
axs[0].set_title('Land-cover')
axs[1].set_title('Reservoirs')

year_start = 1985
year_end = 2019
frame_list = np.concatenate([[year_start]*5, np.arange(year_start, year_end+1), [year_end]*5])
# frame_list = np.arange(year_start, year_end)

src = rio.open('./in/mato_grosso/wgs84/mapbiomas-brazil-collection-90-matogrossomt-1985_1km_mode.tif')
lclayer = rioplot.show(all_cover[1985][0],
             vmin=0, vmax=4,
             cmap=cmap, transform=src.transform, ax=axs[0])
year_df = full_df.loc[full_df['year']==1985]
hexlayer = axs[1].hexbin(year_df['longitude'], year_df['latitude'],
            gridsize=(50),
            vmin=0, vmax=275,
            extent=xlims + ylims,
            cmap='GnBu',
            linewidths=0.1,
            edgecolor='none')
both_layers = [lclayer, hexlayer]

def animate(y):
    year_df = full_df.loc[full_df['year']==y]
    if y != year_start:
        for ax in axs:
            ax.collections[-1].remove()
    both_layers[0] = rioplot.show(all_cover[y][0],
                vmin=0, vmax=4,
                cmap=cmap, transform=src.transform, ax=axs[0])
    both_layers[1] = axs[1].hexbin(year_df['longitude'], year_df['latitude'],
                gridsize=(50),
                vmin=0, vmax=275,
                extent=xlims + ylims,
                cmap='GnBu',
                linewidths=0.1,
                edgecolor='none')
    for ax in axs:
        # Remove outside of mt
        nonmt_poly.plot(ax=ax,color='whitesmoke')
    plt.suptitle(y)
    return both_layers,

ani = FuncAnimation(fig, animate, frames=frame_list, interval=500)
ani.save('../../../../../figs/ch2/frontier_linked.gif')